<H1> Configurações básicas do projeto

In [1]:
import psycopg2
import pandas as pd
import numpy as np
import nltk
import csv
import unidecode
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [2]:
import joblib

In [3]:
dados = pd.read_csv('Base Clientes.csv', on_bad_lines='skip', header = 0, delimiter = ';',encoding='latin-1')
df = pd.DataFrame(dados)
df = df.applymap(lambda x: unidecode.unidecode(str(x).upper()) if pd.notnull(x) else x)
tfidf_vectorizer = TfidfVectorizer()
scaler = StandardScaler()

In [4]:
df = df.drop(['idcnpj_cpf'], axis=1)

In [5]:
df

,sexo,ocupacao,idade,tipo,subtipo,marca
0,1,AUTONOMO,45.0,"{MOVEIS,CELULAR,COLCHOES,COLCHOES}","{""1.60 METROS"",""SMARTPHONE"",""1.38 M"",""1.38 M""}","{""SANTOS ANDIRA"",""SAMSUNG CELULARES"",ECOFLEX,E..."
1,2,APOSENTADO,45.0,{MOVEIS},"{""4 CADEIRAS""}",{FABONE}
2,2,ASSALARIADO,53.0,"{""LINHA BRANCA""}","{""FOGAO PISO 4 BOCAS""}",{CONSUL}
3,1,RURAL,28.0,"{COLCHOES,ELETRO,""LINHA MARROM"",COLCHOES,""LINH...","{""79 CM"",BONECOS,""32 POLEGADAS"",""1.58 M"",""LAVA...","{JS,XICORIA,""PHILCO SOM IMAGEM "",ECOFLEX,CONSU..."
4,1,APOSENTADO,47.0,{CELULAR},"{""SMARTPHONE""}","{""SAMSUNG CELULARES""}"
...,...,...,...,...,...,...
407998,2,RURAL,47.0,"{COLCHOES,COLCHOES,MOVEIS}","{""1.58 M"",""1.38 M"",""6 PORTA""}","{GAZIN,GAZIN,HENN}"
407999,2,APOSENTADO,63.0,"{ELETRO,""LINHA BRANCA""}","{FAQUEIROS,""REF 2 PORTAS CYCLE DEFROST""}","{TRAMONTINA,CONSUL}"
408000,1,OUTRAS,64.0,"{CELULAR,""LINHA BRANCA"",COLCHOES,INFORMATICA,P...","{""SMARTPHONE"",""FOGAO PISO 4 BOCAS"",""1.38 M"",""P...","{""POSITIVO CELULAR"",CONSUL,GAZIN,""MULTILASER C..."
408001,2,ASSALARIADO,53.0,"{""LINHA BRANCA""}","{""FREEZER ATE 300L""}",{BRASTEMP}


----------------------------------------------------------------

<H2> Formatando e adicionando colunas legíveis para vetorização

In [6]:
df['ocupacao_token'] = df['ocupacao'].str.split()
df['tipo_token'] = df['tipo'].str.split()
df['subtipo_token'] = df['subtipo'].str.split()
df['marca_token'] = df['marca'].str.split()

---------------------------------------------

<H2> Vetorizando e transformando cada informação de target em dataframe
    <H5> A coluna de ocupação também será transformada visto que o algoritmo não conseguirá ler no formato de string

In [7]:
ocupacao_token = tfidf_vectorizer.fit_transform(df['ocupacao_token'].apply(lambda x: ''.join(x)))
ocupacao_sp = pd.DataFrame.sparse.from_spmatrix(ocupacao_token, columns=tfidf_vectorizer.get_feature_names_out())
ocupacao_df = pd.DataFrame(ocupacao_sp, columns=tfidf_vectorizer.get_feature_names_out())

In [8]:
tipo_token = tfidf_vectorizer.fit_transform(df['tipo_token'].apply(lambda x: ''.join(x)))
tipo_sp = pd.DataFrame.sparse.from_spmatrix(tipo_token, columns=tfidf_vectorizer.get_feature_names_out())
tipo_df = pd.DataFrame(tipo_sp, columns=tfidf_vectorizer.get_feature_names_out())

In [9]:
subtipo_token = tfidf_vectorizer.fit_transform(df['subtipo_token'].apply(lambda x: ''.join(x)))
subtipo_sp = pd.DataFrame.sparse.from_spmatrix(subtipo_token, columns=tfidf_vectorizer.get_feature_names_out())
subtipo_df = pd.DataFrame(subtipo_sp, columns=tfidf_vectorizer.get_feature_names_out())

In [10]:
marca_token = tfidf_vectorizer.fit_transform(df['marca_token'].apply(lambda x: ''.join(x)))
marca_sp = pd.DataFrame.sparse.from_spmatrix(marca_token, columns=tfidf_vectorizer.get_feature_names_out())
marca_df = pd.DataFrame(marca_sp, columns=tfidf_vectorizer.get_feature_names_out())

--------------------------------

In [11]:
subtipo_token

<408003x422 sparse matrix of type '<class 'numpy.float64'>'
	with 2000018 stored elements in Compressed Sparse Row format>

<h4> Formatando dataframe principal, removendo colunas inutilizadas e concatenando ocupação

In [12]:
df = df.drop(['ocupacao_token','tipo_token','subtipo_token','marca_token','ocupacao','tipo','subtipo','marca'], axis=1)
df = pd.concat([df, ocupacao_df], axis=1)

---------------------------

<H4> Criando dataframes com respectivos targets

In [13]:
df_tipo = pd.concat([df, tipo_df], axis=1)
df_subtipo = pd.concat([df, subtipo_df], axis=1)
df_marca = pd.concat([df, marca_df], axis=1)

-------------------------

<H4> Variável X recebe as features e as demais variáveis recebem os respectivos targets

In [14]:
limite = 80000

In [15]:
X = df.iloc[:, :14][:limite]
y_tipo = df_tipo.iloc[:, 14:][:limite]
y_subtipo = df_subtipo.iloc[:, 14:][:limite]
y_marca = df_marca.iloc[:, 14:][:limite]

------------------

----------------------------

<h2> Criando o modelo

<H4> Separando os dados de treino e teste 

In [16]:
X_tipo_train, X_tipo_test, y_tipo_train, y_tipo_test = train_test_split(X.values, y_tipo.values, test_size=0.3, random_state=501)

C:\Users\pedro-arques\AppData\Local\Temp\ipykernel_24876\3747828516.py:1: FutureWarning: The behavior of .astype from SparseDtype to a non-sparse dtype is deprecated. In a future version, this will return a non-sparse array with the requested dtype. To retain the old behavior, use `obj.astype(SparseDtype(dtype))`
  X_tipo_train, X_tipo_test, y_tipo_train, y_tipo_test = train_test_split(X.values, y_tipo.values, test_size=0.3, random_state=501)


In [17]:
X_subtipo_train, X_subtipo_test, y_subtipo_train, y_subtipo_test = train_test_split(X.values, y_subtipo.values, test_size=0.4, random_state=501)

C:\Users\pedro-arques\AppData\Local\Temp\ipykernel_24876\1706343289.py:1: FutureWarning: The behavior of .astype from SparseDtype to a non-sparse dtype is deprecated. In a future version, this will return a non-sparse array with the requested dtype. To retain the old behavior, use `obj.astype(SparseDtype(dtype))`
  X_subtipo_train, X_subtipo_test, y_subtipo_train, y_subtipo_test = train_test_split(X.values, y_subtipo.values, test_size=0.4, random_state=501)


In [18]:
X_marca_train, X_marca_test, y_marca_train, y_marca_test = train_test_split(X.values, y_marca.values, test_size=0.4, random_state=501)

C:\Users\pedro-arques\AppData\Local\Temp\ipykernel_24876\1656409461.py:1: FutureWarning: The behavior of .astype from SparseDtype to a non-sparse dtype is deprecated. In a future version, this will return a non-sparse array with the requested dtype. To retain the old behavior, use `obj.astype(SparseDtype(dtype))`
  X_marca_train, X_marca_test, y_marca_train, y_marca_test = train_test_split(X.values, y_marca.values, test_size=0.4, random_state=501)


In [87]:
KNN_modelo_tipo = KNeighborsRegressor(n_neighbors=282).fit(X_tipo_train,y_tipo_train)
KNN_predict_tipo = KNN_modelo_tipo.predict(X_tipo_test)

In [88]:
KNN_modelo_subtipo = KNeighborsRegressor(n_neighbors=282).fit(X_subtipo_train,y_subtipo_train)
KNN_predict_subtipo = KNN_modelo_tipo.predict(X_subtipo_test)

In [89]:
KNN_modelo_marca = KNeighborsRegressor(n_neighbors=282).fit(X_marca_train,y_marca_train)
KNN_predict_marca = KNN_modelo_tipo.predict(X_marca_test)

<H4> Calculando RMSE do modelo

------------

<h3> Demonstração do modelo por perfil

In [22]:
X

,sexo,idade,aposentado,assalariado,autonomo,dolar,empresario,estudante,funcionariopublico,militar,outras,pensionista,profissionalliberal,rural
0,1,45.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,45.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,53.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,47.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,2,50.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79996,2,53.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79997,1,43.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79998,1,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [118]:
perfil = np.array(np.array([2,70,1,0,0,0,0,0,0,0,0,0,0,0]))

In [120]:
dictt = {}
count = 0
index = 0
top_tipos = []

y_pred = KNN_modelo_tipo.predict(perfil.reshape(1,-1))

for i in list(y_pred[0]):
    dictt.update({y_tipo.columns[count]:'%.3f' % i})
    count += 1
top_items = sorted(dictt.items(), key=lambda x: x[1], reverse=True)[:10]

for i in top_items:
    (type(top_items[index][0]))
    if float(top_items[index][1]) > 0:
        top_tipos.append(top_items[index][0])
    else:
        pass
    index += 1

In [121]:
dictt = {}
count = 0
index = 0
top_marcas = []

y_pred = KNN_modelo_marca.predict(perfil.reshape(1,-1))

for i in list(y_pred[0]):
    dictt.update({y_marca.columns[count]:'%.3f' % i})
    count += 1
top_items = sorted(dictt.items(), key=lambda x: x[1], reverse=True)[:15]

for i in top_items:
    print(top_items[index][1])
    if float(top_items[index][1]) > 0:
        top_marcas.append(top_items[index][0])
    else:
        pass
    index += 1

0.121
0.103
0.099
0.057
0.053
0.045
0.043
0.040
0.040
0.039
0.035
0.033
0.031
0.029
0.029


In [124]:
dictt = {}
count = 0
index = 0
top_subtipos = []

y_pred = KNN_modelo_subtipo.predict(perfil.reshape(1,-1))

for i in list(y_pred[0]):
    dictt.update({y_subtipo.columns[count]:'%.3f' % i})
    count += 1
top_items = sorted(dictt.items(), key=lambda x: x[1], reverse=True)[:15]

for i in top_items:
    if float(top_items[index][1]) > 0:
        top_subtipos.append(top_items[index][0])
    else:
        pass
    index += 1

In [128]:
dictt = {}
count = 0
index = 0
top_subtipos = []

y_pred = KNN_modelo_subtipo.predict(perfil.reshape(1,-1))

for i in list(y_pred[0]):
    dictt.update({y_subtipo.columns[count]:'%.3f' % i})
    count += 1
top_items = sorted(dictt.items(), key=lambda x: x[1], reverse=True)[:15]

for i in top_items:
    if float(top_items[index][1]) > 0:
        top_subtipos.append(top_items[index][0])
    else:
        pass
    index += 1

In [129]:
top_tipos

['linhabranca',
 'moveis',
 'celular',
 'portatil',
 'eletro',
 'linhamarrom',
 'colchoes',
 'informatica']

In [130]:
dictt

{'0m': '0.000',
 '10porta': '0.007',
 '12porta': '0.003',
 '14a19polegadas': '0.000',
 '14polegadas': '0.002',
 '1lugar': '0.000',
 '1porta': '0.000',
 '20m': '0.024',
 '21polegadas': '0.002',
 '23a24polegadas': '0.005',
 '24polegadas': '0.000',
 '26a28polegadas': '0.000',
 '26polegadas': '0.000',
 '27polegadas': '0.000',
 '28m': '0.001',
 '29polegadas': '0.002',
 '2lugares': '0.009',
 '2porta': '0.002',
 '2portas': '0.000',
 '32polegadas': '0.030',
 '38m': '0.057',
 '39polegadas': '0.003',
 '3lugares': '0.014',
 '3porta': '0.003',
 '3portas': '0.001',
 '3x2lugares': '0.030',
 '40a42polegadas': '0.000',
 '40metros': '0.014',
 '40polegadas': '0.005',
 '42a43polegadas': '0.021',
 '42polegadas': '0.005',
 '43polegadas': '0.000',
 '46a60polegadas': '0.010',
 '46a65polegadas': '0.010',
 '4cadeiras': '0.021',
 '4lugares': '0.000',
 '4porta': '0.011',
 '4portas': '0.000',
 '4x2lugares': '0.000',
 '50m': '0.000',
 '58m': '0.007',
 '5porta': '0.004',
 '60cm': '0.004',
 '60m': '0.003',
 '60metro

In [131]:
pd.DataFrame(top_marcas)

,0
0,samsungcelulares
1,consul
2,britania
3,gazin
4,mondialportateis
5,atlas
6,electrolux
7,colormaq
8,parana
9,positivocelular


In [132]:
top_subtipos

['smartphone',
 'fogaopiso5bocas',
 '38m',
 'liquidificador',
 'celularesfabricantes',
 'lavadorasautomaticaacimade10kg',
 'balcao',
 'ref2portasfrostfree',
 'lavadorassemiautomaticade6a10kg',
 'centrifugas',
 'ventiladormesa',
 'fogaopiso4bocas',
 'polida',
 '32polegadas',
 '3x2lugares']

In [133]:
df2 = pd.concat([pd.DataFrame(top_subtipos), pd.DataFrame(top_marcas),], axis=1)
df2 = df2.applymap(lambda x: unidecode.unidecode(str(x).upper()) if pd.notnull(x) else x)
df2 = df2.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df2.columns = ['subtipo', 'marca']
df3 = pd.DataFrame(top_tipos)
df3 = df3.applymap(lambda x: unidecode.unidecode(str(x).upper()) if pd.notnull(x) else x)
df3.columns = ['tipo']

In [134]:
produtos = pd.read_csv('Produtos 3.0.csv', on_bad_lines='skip', header = 0, delimiter = ';',encoding='latin-1')
df_produtos = pd.DataFrame(produtos)
df_produtos = df_produtos.applymap(lambda x: unidecode.unidecode(str(x).upper()) if pd.notnull(x) else x)
df_produtos['tipo'] = df_produtos['tipo'].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
df_produtos['subtipo'] = df_produtos['subtipo'].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
df_produtos['marca'] = df_produtos['marca'].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)

In [137]:
df1_filtered1 = df_produtos[df_produtos['subtipo'].isin(df2['subtipo'])]
df1_filtered1 = df1_filtered1[df_produtos['marca'].isin(df2['marca'])]
df1_filtered1 = df1_filtered1.reset_index(drop=True)
df1_filtered1

C:\Users\pedro-arques\AppData\Local\Temp\ipykernel_24876\2238021952.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1_filtered1 = df1_filtered1[df_produtos['marca'].isin(df2['marca'])]


,produto,tipo,subtipo,marca,idproduto
0,ESTOFADO PARANA MOSCOU 3X2 LUG,MOVEIS,3X2LUGARES,PARANA,67800
1,FOGAO ELECTROLUX 4 BCS VAPORBAKE FE4IS,LINHABRANCA,FOGAOPISO4BOCAS,ELECTROLUX,72048
2,REFRIGERADOR CONSUL F.FREE CRM56 451L NEW,LINHABRANCA,REF2PORTASFROSTFREE,CONSUL,68672
3,"CENTRIFUGA MUELLER FIT 12,6KG",LINHABRANCA,CENTRIFUGAS,MUELLERLAVADORAS,66756
4,LIQUIDIFICADOR BRITANIA DIAMANTE 800,PORTATIL,LIQUIDIFICADOR,BRITANIA,58403
...,...,...,...,...,...
83,FOGAO ATLAS 4 BCS MONACO TOP GLASS,LINHABRANCA,FOGAOPISO4BOCAS,ATLAS,65737
84,LAVADORA ELECTROLUX LEV17 17KG,LINHABRANCA,LAVADORASAUTOMATICAACIMADE10KG,ELECTROLUX,67001
85,FOGAO ATLAS 4 BCS AGILE UP GLASS,LINHABRANCA,FOGAOPISO4BOCAS,ATLAS,83284
86,VENTILADOR MONDIAL VTX-40-8P 40CM,PORTATIL,VENTILADORMESA,MONDIALPORTATEIS,67429


In [138]:
df2

,subtipo,marca
0,SMARTPHONE,SAMSUNGCELULARES
1,FOGAOPISO5BOCAS,CONSUL
2,38M,BRITANIA
3,LIQUIDIFICADOR,GAZIN
4,CELULARESFABRICANTES,MONDIALPORTATEIS
5,LAVADORASAUTOMATICAACIMADE10KG,ATLAS
6,BALCAO,ELECTROLUX
7,REF2PORTASFROSTFREE,COLORMAQ
8,LAVADORASSEMIAUTOMATICADE6A10KG,PARANA
9,CENTRIFUGAS,POSITIVOCELULAR


In [139]:
# Criar um mapeamento de tipo para ordem
tipo_ordem = {tipo: i for i, tipo in enumerate(df3['tipo'])}

# Ordenar o DataFrame df1_filtered1 pela coluna 'tipo' usando a ordem definida em df3
df1_filtered1_sorted = df1_filtered1.sort_values(by=['tipo'], key=lambda x: x.map(tipo_ordem))

df1_filtered1_sorted

,produto,tipo,subtipo,marca,idproduto
43,FOGAO ATLAS 5 BCS MONACO PLUS,LINHABRANCA,FOGAOPISO5BOCAS,ATLAS,83288
61,LAVADORA ELECTROLUX LEC17 17KG,LINHABRANCA,LAVADORASAUTOMATICAACIMADE10KG,ELECTROLUX,66375
31,FOGAO ATLAS 5 BCS AGILE UP GLASS NEW,LINHABRANCA,FOGAOPISO5BOCAS,ATLAS,68919
32,LAVADORA MUELLER FAMILY LITE 10KG,LINHABRANCA,LAVADORASSEMIAUTOMATICADE6A10KG,MUELLERLAVADORAS,65192
33,REFRIGERADOR ELECTROLUX IM8S 590L,LINHABRANCA,REF2PORTASFROSTFREE,ELECTROLUX,67209
...,...,...,...,...,...
64,VENTILADOR MONDIAL REPELENTE 40CM VTX-40-8P-RL,PORTATIL,VENTILADORMESA,MONDIALPORTATEIS,68730
54,VENTILADOR MONDIAL SUPER POWER 30CM VSP-30-B,PORTATIL,VENTILADORMESA,MONDIALPORTATEIS,66382
30,LIQUIDIFICADOR PHILCO PH900 FILTRO,PORTATIL,LIQUIDIFICADOR,PHILCOPORTATEIS,51968
87,LIQUIDIFICADOR MONDIAL EASY POWER L-550W,PORTATIL,LIQUIDIFICADOR,MONDIALPORTATEIS,65728


In [140]:
df2

,subtipo,marca
0,SMARTPHONE,SAMSUNGCELULARES
1,FOGAOPISO5BOCAS,CONSUL
2,38M,BRITANIA
3,LIQUIDIFICADOR,GAZIN
4,CELULARESFABRICANTES,MONDIALPORTATEIS
5,LAVADORASAUTOMATICAACIMADE10KG,ATLAS
6,BALCAO,ELECTROLUX
7,REF2PORTASFROSTFREE,COLORMAQ
8,LAVADORASSEMIAUTOMATICADE6A10KG,PARANA
9,CENTRIFUGAS,POSITIVOCELULAR


In [141]:
tipo_ordem

{'LINHABRANCA': 0,
 'MOVEIS': 1,
 'CELULAR': 2,
 'PORTATIL': 3,
 'ELETRO': 4,
 'LINHAMARROM': 5,
 'COLCHOES': 6,
 'INFORMATICA': 7}

In [142]:
df1_filtered1_sorted

,produto,tipo,subtipo,marca,idproduto
43,FOGAO ATLAS 5 BCS MONACO PLUS,LINHABRANCA,FOGAOPISO5BOCAS,ATLAS,83288
61,LAVADORA ELECTROLUX LEC17 17KG,LINHABRANCA,LAVADORASAUTOMATICAACIMADE10KG,ELECTROLUX,66375
31,FOGAO ATLAS 5 BCS AGILE UP GLASS NEW,LINHABRANCA,FOGAOPISO5BOCAS,ATLAS,68919
32,LAVADORA MUELLER FAMILY LITE 10KG,LINHABRANCA,LAVADORASSEMIAUTOMATICADE6A10KG,MUELLERLAVADORAS,65192
33,REFRIGERADOR ELECTROLUX IM8S 590L,LINHABRANCA,REF2PORTASFROSTFREE,ELECTROLUX,67209
...,...,...,...,...,...
64,VENTILADOR MONDIAL REPELENTE 40CM VTX-40-8P-RL,PORTATIL,VENTILADORMESA,MONDIALPORTATEIS,68730
54,VENTILADOR MONDIAL SUPER POWER 30CM VSP-30-B,PORTATIL,VENTILADORMESA,MONDIALPORTATEIS,66382
30,LIQUIDIFICADOR PHILCO PH900 FILTRO,PORTATIL,LIQUIDIFICADOR,PHILCOPORTATEIS,51968
87,LIQUIDIFICADOR MONDIAL EASY POWER L-550W,PORTATIL,LIQUIDIFICADOR,MONDIALPORTATEIS,65728


In [143]:
resultado = df1_filtered1_sorted.loc[df1_filtered1_sorted['tipo'] == 'CELULAR']

In [144]:
df_filtrado = df1_filtered1_sorted.groupby('subtipo').head(2)

In [145]:
df_filtrado

,produto,tipo,subtipo,marca,idproduto
43,FOGAO ATLAS 5 BCS MONACO PLUS,LINHABRANCA,FOGAOPISO5BOCAS,ATLAS,83288
61,LAVADORA ELECTROLUX LEC17 17KG,LINHABRANCA,LAVADORASAUTOMATICAACIMADE10KG,ELECTROLUX,66375
31,FOGAO ATLAS 5 BCS AGILE UP GLASS NEW,LINHABRANCA,FOGAOPISO5BOCAS,ATLAS,68919
32,LAVADORA MUELLER FAMILY LITE 10KG,LINHABRANCA,LAVADORASSEMIAUTOMATICADE6A10KG,MUELLERLAVADORAS,65192
33,REFRIGERADOR ELECTROLUX IM8S 590L,LINHABRANCA,REF2PORTASFROSTFREE,ELECTROLUX,67209
73,REFRIGERADOR CONSUL F.FREE CRM50 410L NEW,LINHABRANCA,REF2PORTASFROSTFREE,CONSUL,68670
72,CENTRIFUGA MUELLER FIT C/ TIMER 15KG,LINHABRANCA,CENTRIFUGAS,MUELLERLAVADORAS,68783
70,LAVADORA CONSUL CWH15 NEW 15KG,LINHABRANCA,LAVADORASAUTOMATICAACIMADE10KG,CONSUL,67200
38,FOGAO ELECTROLUX 4 BCS VAPORBAKE FE4GP,LINHABRANCA,FOGAOPISO4BOCAS,ELECTROLUX,72072
47,LAVADORA COLORMAQ LCS10 10KG,LINHABRANCA,LAVADORASSEMIAUTOMATICADE6A10KG,COLORMAQ,64652


In [ ]:
joblib.dump(KNN_modelo_tipo, 'modelo_tipo.pkl')

In [ ]:
joblib.dump(KNN_modelo_subtipo, 'modelo_subtipo.pkl')

In [ ]:
joblib.dump(KNN_modelo_marca, 'modelo_marca.pkl')

In [ ]:
count = 0
for i in list(y_pred[0]):
    dictt.update({y_tipo.columns[count]:'%.3f' % i})
    count += 1
top_items = sorted(dictt.items(), key=lambda x: x[1], reverse=True)[:10]


In [ ]:
y_tipo.to_excel('dados.xlsx', index=False)

In [ ]:
y_tipo

In [ ]:
print(len(y_pred[0]))
print(len(y_tipo))

In [ ]:
y_tipo

In [ ]:
y_subtipo.columns[0]

In [ ]:
for i in list(y_pred[0]):
    dictt.update({y_subtipo.columns[count]:'%.3f' % i})
    count += 1
top_items = sorted(dictt.items(), key=lambda x: x[1], reverse=True)[:15]

In [ ]:
len(y_pred[0])

In [ ]:
y_subtipo.columns

In [ ]:
y_subtipo = pd.DataFrame(y_subtipo.columns)
y_tipo = pd.DataFrame(y_tipo.columns)
y_marca = pd.DataFrame(y_marca.columns)

In [ ]:
y_marca

In [ ]:
y_subtipo.to_csv('y_subtipo df.csv', index=False)
y_tipo.to_csv('y_tipo df.csv', index=False)
y_marca.to_csv('y_marca df.csv', index=False)

In [ ]:
y_tipo[0]

In [ ]:
y_subtipo

In [ ]:
y_pred

In [ ]:
def top_marcas():
    dictt = {}
    count = 0
    index = 0
    top_marcas = []
    for i in list(y_pred_marca[0]):
        dictt.update({y_marca.iloc[count].item(): i})
        count += 1
    top_marcas = sorted(dictt.items(), key=lambda x: x[1], reverse=True)[:15]

    return top_marcas
top_marcas()

In [ ]:
top_subtipos()

In [24]:
print(t[2][1:4])

(1, None, 29)


In [26]:
perfil = ('1,50,0,0,0,0,0,0,0,0,0,0,0,0')

NameError: name 'perfil' is not defined

In [60]:
# Criar um mapeamento de tipo para ordem
tipo_ordem = {tipo: i for i, tipo in enumerate(df3['tipo'])}

# Ordenar o DataFrame df1_filtered1 pela coluna 'tipo' usando a ordem definida em df3
df1_filtered1_sorted = df1_filtered1.sort_values(by=['tipo'], key=lambda x: x.map(tipo_ordem))

df1_filtered1_sorted

,produto,tipo,subtipo,marca,idproduto
85,ARMARIO HENN CONNECT 1PT 35CM,MOVEIS,ARMARIOS,HENN,64377
83,G ROUPA HENN ATALAIA 6PT 6GV C/ ESP,MOVEIS,6PORTA,HENN,68877
81,ARMARIO INDEKES MONZA 1PT 60CM 61726,MOVEIS,ARMARIOS,INDEKES,67604
128,G ROUPA HENN BRIZ 6PT 6GAV C/ ESP,MOVEIS,6PORTA,HENN,68028
78,"ARMARIO INDEKES MONZA 2PT VD REFL 1,20CM 617241.",MOVEIS,ARMARIOS,INDEKES,67607
...,...,...,...,...,...
82,SUPORTE ELG ARTICULADO TV 26'' ATE 55'' FULL 4...,LINHAMARROM,SUPORTESDEPAREDE,ELG,65480
100,SUPORTE FIXO ELG TV UNI100 UNIVERSAL,LINHAMARROM,SUPORTESDEPAREDE,ELG,56892
27,SUPORTE ELG FIXO DE PAREDE UNI88,LINHAMARROM,SUPORTESDEPAREDE,ELG,68431
79,"SUPORTE ELG FIXO TVS E MONITORES 32"" ATE 77""",LINHAMARROM,SUPORTESDEPAREDE,ELG,67635


In [86]:
# Criar um mapeamento de tipo para ordem
tipo_ordem = {tipo: i for i, tipo in enumerate(df3['tipo'])}

# Ordenar o DataFrame df1_filtered1 pela coluna 'tipo' usando a ordem definida em df3
df1_filtered1_sorted = df1_filtered1.sort_values(by=['tipo'], key=lambda x: x.map(tipo_ordem))

df1_filtered1_sorted

,produto,tipo,subtipo,marca,idproduto
0,ESTOFADO PARANA MOSCOU 3X2 LUG,MOVEIS,3X2LUGARES,PARANA,67800
70,ESTOFADO PARANA PARIS 3X2 LUG,MOVEIS,3X2LUGARES,PARANA,64783
69,ESTOFADO PARANA SANDERO 3X2 LUG,MOVEIS,3X2LUGARES,PARANA,55879
68,ES. BEM MM MONTANA 3X2 LUG,MOVEIS,3X2LUGARES,PARANA,64431
75,ESTOFADO PARANA FLUENCE 3X2 LUG,MOVEIS,3X2LUGARES,PARANA,48257
...,...,...,...,...,...
7,VENTILADOR BRITANIA TURBO 40CM BVT440,PORTATIL,VENTILADORMESA,BRITANIA,68727
3,LIQUIDIFICADOR BRITANIA DIAMANTE 800,PORTATIL,LIQUIDIFICADOR,BRITANIA,58403
73,LIQUIDIFICADOR MONDIAL TURBO POWER L-99-FB,PORTATIL,LIQUIDIFICADOR,MONDIALPORTATEIS,66206
90,LIQUIDIFICADOR MONDIAL EASY POWER L-550W,PORTATIL,LIQUIDIFICADOR,MONDIALPORTATEIS,65728


In [117]:
# Criar um mapeamento de tipo para ordem
tipo_ordem = {tipo: i for i, tipo in enumerate(df3['tipo'])}

# Ordenar o DataFrame df1_filtered1 pela coluna 'tipo' usando a ordem definida em df3
df1_filtered1_sorted = df1_filtered1.sort_values(by=['tipo'], key=lambda x: x.map(tipo_ordem))

df1_filtered1_sorted

,produto,tipo,subtipo,marca,idproduto
0,ESTOFADO PARANA MOSCOU 3X2 LUG,MOVEIS,3X2LUGARES,PARANA,67800
68,ESTOFADO PARANA PARIS 3X2 LUG,MOVEIS,3X2LUGARES,PARANA,64783
67,ESTOFADO PARANA SANDERO 3X2 LUG,MOVEIS,3X2LUGARES,PARANA,55879
66,ES. BEM MM MONTANA 3X2 LUG,MOVEIS,3X2LUGARES,PARANA,64431
56,ESTOFADO PARANA NOBILE 3X2 LUG,MOVEIS,3X2LUGARES,PARANA,67031
...,...,...,...,...,...
3,LIQUIDIFICADOR BRITANIA DIAMANTE 800,PORTATIL,LIQUIDIFICADOR,BRITANIA,58403
71,LIQUIDIFICADOR MONDIAL TURBO POWER L-99-FB,PORTATIL,LIQUIDIFICADOR,MONDIALPORTATEIS,66206
88,LIQUIDIFICADOR MONDIAL EASY POWER L-550W,PORTATIL,LIQUIDIFICADOR,MONDIALPORTATEIS,65728
47,"TV 32"" PHILCO SMART HD ANDROID TV LEDPTV32G23A...",LINHAMARROM,32POLEGADAS,PHILCOSOMIMAGEM,68823


In [146]:
# Criar um mapeamento de tipo para ordem
tipo_ordem = {tipo: i for i, tipo in enumerate(df3['tipo'])}

# Ordenar o DataFrame df1_filtered1 pela coluna 'tipo' usando a ordem definida em df3
df1_filtered1_sorted = df1_filtered1.sort_values(by=['tipo'], key=lambda x: x.map(tipo_ordem))

df1_filtered1_sorted

,produto,tipo,subtipo,marca,idproduto
43,FOGAO ATLAS 5 BCS MONACO PLUS,LINHABRANCA,FOGAOPISO5BOCAS,ATLAS,83288
61,LAVADORA ELECTROLUX LEC17 17KG,LINHABRANCA,LAVADORASAUTOMATICAACIMADE10KG,ELECTROLUX,66375
31,FOGAO ATLAS 5 BCS AGILE UP GLASS NEW,LINHABRANCA,FOGAOPISO5BOCAS,ATLAS,68919
32,LAVADORA MUELLER FAMILY LITE 10KG,LINHABRANCA,LAVADORASSEMIAUTOMATICADE6A10KG,MUELLERLAVADORAS,65192
33,REFRIGERADOR ELECTROLUX IM8S 590L,LINHABRANCA,REF2PORTASFROSTFREE,ELECTROLUX,67209
...,...,...,...,...,...
64,VENTILADOR MONDIAL REPELENTE 40CM VTX-40-8P-RL,PORTATIL,VENTILADORMESA,MONDIALPORTATEIS,68730
54,VENTILADOR MONDIAL SUPER POWER 30CM VSP-30-B,PORTATIL,VENTILADORMESA,MONDIALPORTATEIS,66382
30,LIQUIDIFICADOR PHILCO PH900 FILTRO,PORTATIL,LIQUIDIFICADOR,PHILCOPORTATEIS,51968
87,LIQUIDIFICADOR MONDIAL EASY POWER L-550W,PORTATIL,LIQUIDIFICADOR,MONDIALPORTATEIS,65728
